In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import skimage.io
import tensorflow 
import os
import glob
import tqdm

from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,InputLayer, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D

%matplotlib inline 

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
print("Tensorflow version: ",tf.__version__)

In [1]:
# Training Data

Low_quality_Apple = os.listdir('FruitsDB/Train/Low_quality_Apple/')
Low_quality_mango    = os.listdir('FruitsDB/Train/Low_quality_mango/')
Quality_Apples     = os.listdir('FruitsDB/Train/Quality_Apples/')
Quality_Mangoes      = os.listdir('FruitsDB/Train/Quality_Mangoes/')

NameError: name 'os' is not defined

In [5]:

train_dir = 'FruitsDB/Train'
test_dir = 'FruitsDB/Test'

image_size = 224

In [6]:
batch_size = 32

train_datagen = ImageDataGenerator(rescale = 1./255,
                            rotation_range=45,
                            zoom_range=0.40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.15,
                            horizontal_flip=True,
                            vertical_flip= True,
                            fill_mode="nearest")

train_data = train_datagen.flow_from_directory(train_dir,
                                              target_size=(150,150),
                                              batch_size=32,
                                              class_mode="categorical")


Found 188 images belonging to 4 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(test_dir,
                                            target_size=(150,150),
                                            batch_size=32,
                                            class_mode="categorical")

Found 60 images belonging to 4 classes.


In [8]:

#=================================================================
chanDim = 1
model = Sequential(name="FruitNet")
model.add(Conv2D(32, (3, 3), padding="same",input_shape=(150,150,3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding="same"))
model.add(Activation("relu"))

model.add(Conv2D(64, (4, 4), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (4, 4), padding="same"))
model.add(Activation("relu"))

model.add(Conv2D(128, (4, 4), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation("softmax"))


model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
reduction_lr = ReduceLROnPlateau(monitor = "val_accuracy",patience = 2 ,verbose = 1, factor = 0.2, min_lr = 0.00001)
model.summary()
#plot_model(model,show_shapes=True)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "FruitNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 64)        32832     
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 64)        0         
___________________________________________

In [9]:
callback_list = [reduction_lr]

In [10]:
#bot_callback = botCallback(access_token)
#plotter = Plotter(access_token)
#callback_list = [bot_callback,plotter] callbacks=callback_list
#start = time.time()

history = model.fit_generator(train_data,
                    steps_per_epoch = 30,
                    validation_data = test_data,
                    validation_steps = 30,
                    epochs=20,
                    callbacks = callback_list)
#end = time.time()
#print("Total train time: ",(end-start)/60," mins")

Epoch 1/20
30/30 [==============================] - 40s 1s/step - loss: 1.1135 - acc: 0.7596 - val_loss: 11.3103 - val_acc: 0.4333
Epoch 2/20
30/30 [==============================] - 39s 1s/step - loss: 0.5889 - acc: 0.8138 - val_loss: 4.8739 - val_acc: 0.4000
Epoch 3/20
30/30 [==============================] - 42s 1s/step - loss: 0.5738 - acc: 0.8511 - val_loss: 3.0678 - val_acc: 0.1833
Epoch 4/20
30/30 [==============================] - 40s 1s/step - loss: 0.4154 - acc: 0.8798 - val_loss: 3.4043 - val_acc: 0.4667
Epoch 5/20
30/30 [==============================] - 42s 1s/step - loss: 0.4706 - acc: 0.8766 - val_loss: 4.3183 - val_acc: 0.2000
Epoch 6/20
30/30 [==============================] - 41s 1s/step - loss: 0.3016 - acc: 0.9106 - val_loss: 0.4114 - val_acc: 0.8833
Epoch 7/20
30/30 [==============================] - 40s 1s/step - loss: 0.2900 - acc: 0.9245 - val_loss: 1.3070 - val_acc: 0.6333
Epoch 8/20
30/30 [==============================] - 41s 1s/step - loss: 0.2563 - acc: 0.9

Epoch 19/20
30/30 [==============================] - 43s 1s/step - loss: 0.1833 - acc: 0.9500 - val_loss: 0.0732 - val_acc: 0.9667
Epoch 20/20
30/30 [==============================] - 46s 2s/step - loss: 0.1464 - acc: 0.9489 - val_loss: 0.2272 - val_acc: 0.9167


In [11]:
def plot_graph(history,string):
    plt.figure(figsize=(12,8))
    plt.plot(history.history[string],label=str(string))
    plt.plot(history.history["val_"+str(string)],label="val_"+str(string))
    plt.xlabel("Epochs")
    plt.ylabel(str(string))
    plt.legend()
    plt.show()
plot_graph(history,"accuracy")
plot_graph(history,"loss")

KeyError: 'accuracy'

<Figure size 864x576 with 0 Axes>

In [12]:
model.save("FruitNet.h5")